In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using Plots
using Test
include("car.jl");
using ForwardDiff

  Activating environment at `c:\Users\ADMIN\Documents\GitHub\ocrl_project\Project.toml`
  Activating environment at `c:\Users\ADMIN\Documents\GitHub\ocrl_project\Project.toml`


In [2]:
# Define the model
model = BicycleModel()

# get the number of states and controls
n = state_dim(model)
m = control_dim(model)

# Evaluate the continuous and discrete dynamics
x0 = SA[0.,0.,0.,0.]
u0 = SA[0.,0.]
t0 = 0.0
dt = 0.025
dynamics(model, x0, u0)
discrete_dynamics(RK4, model, x0, u0, t0, dt)  # use rk4 for integration

# Evaluate the continuous and discrete Jacobians
z0 = KnotPoint(x0,u0,dt,t0)   # create a `KnotPoint` type that stores everything together
∇f = RobotDynamics.DynamicsJacobian(model)
jacobian!(∇f, model, z0)
discrete_jacobian!(RK4, ∇f, model, z0)

# Extract pieces of the Jacobian
A = ∇f.A
B = ∇f.B;

In [3]:
nx = n
nu = m

Q = Diagonal([1,1,1e-2,1e-2])
R = Diagonal([1e-1,1e-1])
Qf = Diagonal([1,1,1,1.])*10

N = 101 #401
Nmpc = 10 #60
Nhz = 1
delta = 12 #80
Nc = N + 2*delta

FD = ForwardDiff;

In [10]:
function get__vanilla_trajectory(s)
    
    arr = split(s,"")
    if 60 < length(arr) < 120
        c = 2
    elseif length(arr) > 120
        c = 1
    else 
        c = 3
    end 

    N = length(arr) * c
   

    x1ref = zeros(N)
    x2ref = zeros(N)
    θref = zeros(N)
    v1ref = zeros(N)
    v2ref = zeros(N)
    ωref = zeros(N)
    rad = 20

    u1ref = zeros(N-1)*NaN
    u2ref = zeros(N-1)*NaN

    #v1ref[1] = 0.0
    #v1ref[Nc] = v1ref[Nc-1]
    @show(size(θref))
    counter = 0
    for i in 1:length(arr)
        
        if arr[i] == "s"
            x1ref[i] += 0
            x2ref[i] += 0
            θref[i] += 0
        elseif arr[i] == "w"
            x2ref[i] +=1
            
        elseif arr[i] == "a"
            x1ref[i] += 1
        elseif arr[i] =="d"
            x1ref[i] -=1
        end
        
    end
    xref = [x1ref'; x2ref'; θref'; v1ref']
    uref = [u1ref'; u2ref']
    return [x for x in eachcol(xref)], [u for u in eachcol(uref)], length([x for x in eachcol(xref)])
end

get__vanilla_trajectory (generic function with 1 method)

In [85]:
using Interpolations
using Plots

In [190]:
function Linear_Fit_between(knot1,knot2,steps)
    if steps == 0
        return 0
    end

    if steps == 1
        return [knot1]
    end

    delta = (knot2 - knot1)/(steps)
    #@show(delta)
    array = zeros(steps)
    #@show(size(array))
    array[1] = knot1
    for i in 1:steps-1
        array[i+1] = array[i] + delta
    end
    return array
end

Linear_Fit_between (generic function with 1 method)

In [211]:
function get_complex_traj(input)
    arr = split(input,"")
    #println(arr)
    if 60 < length(arr) < 120
        c = 2
    elseif length(arr) > 120
        c = 1
    else 
        c = 3
    end

    c = c*16
    N = ((length(arr)÷ 4))*c 
    @show N 
    if N == 0
        return zeros(12,4)
    end
    
    x1ref = zeros(N)*NaN # x-Coord
    x2ref = zeros(N)*NaN # y-Coord
    θref = zeros(N)*NaN # Angular (tan-1(y/x))
    v1ref = zeros(N)*NaN
    v2ref = zeros(N)*NaN
    ωref = zeros(N)*NaN
    rad = 20

    u1ref = zeros(N-1)
    u2ref = zeros(N-1)

    block = zeros((length(arr) ÷ 4),4)*NaN
    #@show size(block)
    
    counter  = 1
    row_count = 1

    xc = 0.0
    yc = 0.0
    tc = 0.0
    vc = 0.0

    Nb,Cb = size(block)
    #@show(Nb)

    for i in 1:length(arr)
        #@show(xc)
        if arr[i] == "s"
            continue
        elseif arr[i] == "w"
            xc = xc + 0.0
            yc = yc + 1.0
        elseif arr[i] == "a"
            xc = xc - 1.0
            yc = yc + 0.0
        elseif arr[i] == "d"
            xc = xc + 1.0
            yc = yc + 0.0
        elseif arr[i] == "q"
            xc -=1.0
            yc +=1.0
        elseif arr[i] == "e"
            xc = xc + 1.0
            yc = yc + 1.0
        end
        #@show(yc)

        
        if counter == 4
            counter = 0
            
            block[row_count,1] = xc
            block[row_count,2] = yc
            block[row_count,3] = tc
            block[row_count,4] = vc

            if row_count > Nb 
                break
            end
            
            row_count +=1
            #@show(row_count)
        end
        
        counter +=1
    end
    #@show(block)
    
    start_vec = zeros(1,4)
    x1ref[1:c] .= Linear_Fit_between(0,block[1,1],c)
    x2ref[1:c] .= Linear_Fit_between(0,block[1,2],c)

    for i = 2:Nb
        start = (i-1)*c + 1
        finish = i*c
        #@show(c)
        #@show(finish)
        #@show block[i,1]
        #@show block[i+1,1]
        arrx = Linear_Fit_between(block[i-1,1],block[i,1],c)
        arry = Linear_Fit_between(block[i-1,2],block[i,2],c)
        #@show size(arrx)

        x1ref[start:finish] .= arrx
        x2ref[start:finish] .= arry
    end
    x1ref[end] = block[Nb,1]
    x2ref[end] = block[Nb,2]
    #@show x1ref

    #@show size(x1ref)
    θref[1] = 0
    v1ref[1] = 0
    for i = 2:N
        θref[i] = atan(x2ref[i]-x2ref[i-1])/(x1ref[i]-x1ref[i-1])
        v1ref[i] = sqrt((x2ref[i]-x2ref[i-1])^2 + (x1ref[i]-x1ref[i-1])^2)
    end

    xref = [x1ref'; x2ref'; θref'; v1ref']
    uref = [u1ref'; u2ref']
    #@show size([x for x in eachcol(xref)])
    return [x for x in eachcol(xref)], [u for u in eachcol(uref)]
end

get_complex_traj (generic function with 1 method)

In [212]:
in = readline()
get_complex_traj(in)

N = 528


(SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}[[0.0, 0.0, 0.0, 0.0], [0.0, 0.08333333333333333, Inf, 0.08333333333333333], [0.0, 0.16666666666666666, Inf, 0.08333333333333333], [0.0, 0.25, Inf, 0.08333333333333334], [0.0, 0.3333333333333333, Inf, 0.08333333333333331], [0.0, 0.41666666666666663, Inf, 0.08333333333333331], [0.0, 0.49999999999999994, Inf, 0.08333333333333331], [0.0, 0.5833333333333333, Inf, 0.08333333333333331], [0.0, 0.6666666666666666, Inf, 0.08333333333333337], [0.0, 0.75, Inf, 0.08333333333333337]  …  [23.16666666666662, 22.375, 0.7490257199514988, 0.10416666666666571], [23.249999999999954, 22.4375, 0.7490257199514988, 0.10416666666666571], [23.333333333333286, 22.5, 0.7490257199514988, 0.10416666666666571], [23.416666666666618, 22.5625, 0.7490257199514988, 0.10416666666666571], [23.49999999999995, 22.625, 0.7490257199514988, 0.10416666666666571], [23.583333333333282, 22.6875, 0.7490257199514988, 0.10416666666666571], [23.666